In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import ADASYN
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score

In [0]:
data=pd.read_csv('NORMALIZADO.csv')

In [0]:
cols=['Age','BysunessTrave','DailyRate','Department','DistanceFromHome', 'Education', 'EducationField','EmployeeNumber','EnvironmentSatifaction', 'Gender', 'HourlyRate', 'Joblnvolvement', 'JobLevel'
      'JobRole','JobSatisfaction','MaritalStatus','MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel'
      'TotalWorkingYears', 'TraininTimesLastYear', 'WorkLifeBalance','YearAtCompany', 'YearInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager','Attrition']

In [0]:
Datos=data.iloc[:,:-1].values#todas menos la ultima
Categoria=data.iloc[:,-1].values #ultima columna

In [0]:
from collections import Counter

In [0]:
Counter(Categoria)

In [0]:
ada= ADASYN(sampling_strategy='minority', random_state=4,n_neighbors=1)

In [0]:
DatosNorma, CategoriaNorma =ada.fit_resample(Datos,Categoria)

In [0]:
print('OVersampled Target variable distribucion;', Counter(CategoriaNorma))

In [0]:
########################################################RANDOM FOREST#############################################

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(DatosNorma, CategoriaNorma, test_size=0.3,)

In [0]:
####################hiperparametros##############

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
####cuadrilla de parametros#######

In [0]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

In [0]:
##############cuadrilla aleatoria############

In [0]:
random_grid = { 'n_estimators': n_estimators, 
               'max_features': max_features, 
               'max_depth': max_depth, 
               'min_samples_split': min_samples_split, 
               'min_samples_leaf': min_samples_leaf, 
               'bootstrap': bootstrap} 

In [0]:
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [0]:
clf=RandomForestClassifier()

In [0]:
from sklearn.model_selection import RandomizedSearchCV 

In [0]:
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, n_jobs = -1)

In [27]:
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [29]:
print (rf_random.best_params_)
rf_random.best_score_

{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 80, 'bootstrap': True}


0.9239766081871346

In [30]:
print("precisión entranamiento: {0: .2f}".format(
       rf_random.score(X_train, y_train)))

precisión entranamiento:  1.00


In [0]:
################################VALIDACION CRUZADA##################

In [0]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [35]:
resultado= cross_val_score(rf_random,DatosNorma,CategoriaNorma, cv=5)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   48.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.2min finished


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   58.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.7min finished


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.8min finished


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.2min finished


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.0min finished


In [36]:
print(resultado)

[0.74829932 0.89795918 0.91156463 0.95238095 0.9109589 ]


In [40]:
print (resultado.mean())

0.8842325971484485


In [0]:
#################prueba###########################

In [0]:
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
y_pred=rf_random.predict(X_test)

In [0]:
from sklearn import metrics

In [46]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9095022624434389


In [0]:
from sklearn.metrics import classification_report, confusion_matrix

In [50]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[107   9]
 [ 11  94]]
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       116
           1       0.91      0.90      0.90       105

    accuracy                           0.91       221
   macro avg       0.91      0.91      0.91       221
weighted avg       0.91      0.91      0.91       221

